In [1]:
import pandas as pd
from data_cleaning.text_to_numeric import get_zimmer, get_stockwerk, get_heizung, get_kindergarten, get_schule, get_bahn, get_miete, get_nebenkosten, get_alter, get_lage, get_kaution, get_kueche, get_bad, get_mobliert, get_quadratmeter
from combined_mass.combined_mass import calculate_combined_mass

In [2]:
file_path = '../../data/Wohnungen_1.csv'  # Gegebenen Datensatz einlesen
data = pd.read_csv(file_path, sep=';')

# Bewohnerkategorie bestimmen
bewohnerkategorie = ['Studierende', 'Kleinfamilie', 'DINK', 'Alleinerziehende', 'Expatriate', 'Rentnerpaar']

data['Bewohnerkategorie'] = data[bewohnerkategorie].apply(lambda row: 'Keine' if all(row == 'nein') else ', '.join(row[row == 'ja'].index), axis=1)

# Bereinigung der Daten
data = data[data['Bewohnerkategorie'] != 'Keine']

# Spalten, in denen "nein" durch 0 und "ja" durch 1 ersetzt werden sollen
columns_to_replace = ["Hausmeister", "Garage", "Aufzug", "Balkon", "Terrasse", "Kehrwoche"]

# Ersetze "nein" und "ja" in den angegebenen Spalten
data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})

data['Zimmerzahl'] = data['Zimmerzahl'].apply(get_zimmer)
data['Stockwerk'] = data['Stockwerk'].apply(get_stockwerk)
data['Heizung'] = data['Heizung'].apply(get_heizung)
data['Kindergarten'] = data['Kindergarten'].apply(get_kindergarten)
data['Schule'] = data['Schule'].apply(get_schule)
data['S-Bahn'] = data['S-Bahn'].apply(get_bahn)
data['Miete'] = data['Miete'].apply(get_miete)
data['Nebenkosten'] = data['Nebenkosten'].apply(get_nebenkosten)
data['Alter'] = data['Alter'].apply(get_alter)
data['Lage'] = data['Lage'].apply(get_lage)
data['Kaution'] = data['Kaution'].apply(get_kaution)
data['Kueche'] = data['Kueche'].apply(get_kueche)
data['Bad'] = data['Bad'].apply(get_bad)
data['Moebliert'] = data['Moebliert'].apply(get_mobliert)
data['Quadratmeter'] = data['Quadratmeter'].apply(get_quadratmeter)

print(data.head())

    Zimmerzahl  Stockwerk  Heizung  Hausmeister  Kindergarten  Schule  S-Bahn  \
11           2          0        0            0             1       2       4   
13           2          0        5            0             2       2       3   
23           2          1        4            1             1       2       4   
27           0          0        1            0             1       0       0   
40           0          8        5            0             0       2       2   

    Garage  Miete  Nebenkosten  ...  Kehrwoche  Moebliert  Quadratmeter  \
11       0    726          276  ...          0          0           116   
13       1    726          226  ...          0          0           106   
23       0    776          276  ...          0          0           116   
27       0    226           50  ...          0          0            25   
40       0    226           50  ...          0          0            36   

    Studierende  Kleinfamilie  DINK  Alleinerziehende  Expatri

/tmp/ipykernel_2961/1043316277.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})


In [3]:
omega = ['Studierende', 'Kleinfamilie', 'DINK', 'Alleinerziehende', 'Expatriate', 'Rentnerpaar']

In [21]:
# mass_hausmeister = calculate_basismaße_multivalue(data, 'Hausmeister', omega)
# print("Basismaße für Hausmeister:", mass_hausmeister)

Basismaße für Garage: {set():0.7164179104477613; {'Studierende'}:0.208955223880597; {'Kleinfamilie'}:0.07462686567164178; {'DINK'}:0.0; {'Alleinerziehende'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}


In [3]:
# Kombiniere Basismaße (z. B. Garage und Zimmerkategorie)
# combined_mass = mass_garage & mass_aufzug
# print("Kombinierte Evidenz:", combined_mass)
# Wahrscheinlichste Hypothese auswählen
# most_likely = combined_mass.max_bel()
# print("Am wahrscheinlichsten ist:", most_likely)

In [17]:
# Gegebene Attribut-Wert-Kombinationen\n",
attributes_values = {
    'Zimmerzahl': 1,
    'Stockwerk': 0,
}

# Berechne die kombinierte Evidenz\n",
combined_mass = calculate_combined_mass(data, attributes_values, omega)

# Ausgabe der kombinierten Evidenz
print("Kombinierte Evidenz:", combined_mass)

# Wahrscheinlichste Hypothese basierend auf der kombinierten Evidenz
most_likely = combined_mass.max_bel()  # Maximale Glaubwürdigkeit

print("Am wahrscheinlichsten ist:", most_likely)

Basismaße für Zimmerzahl=1: {set():0.9850746268656716; {'Expatriate'}:0.014925373134328358; {'Studierende'}:0.0; {'Kleinfamilie'}:0.0; {'DINK'}:0.0; {'Alleinerziehende'}:0.0; {'Rentnerpaar'}:0.0}
Basismaße für Stockwerk=0: {set():0.7313432835820896; {'Kleinfamilie'}:0.19402985074626866; {'Studierende'}:0.05970149253731343; {'Expatriate'}:0.014925373134328358; {'DINK'}:0.0; {'Alleinerziehende'}:0.0; {'Rentnerpaar'}:0.0}
Kombinierte Evidenz: {{'Expatriate'}:1.0; {'Studierende'}:0.0; {'Kleinfamilie'}:0.0; {'DINK'}:0.0; {'Alleinerziehende'}:0.0; {'Rentnerpaar'}:0.0}
Kombinierte Evidenz: {{'Expatriate'}:1.0; {'Studierende'}:0.0; {'Kleinfamilie'}:0.0; {'DINK'}:0.0; {'Alleinerziehende'}:0.0; {'Rentnerpaar'}:0.0}
Am wahrscheinlichsten ist: frozenset({'Expatriate'})
